# Revealed comparative advantage

This notebook computes revealed comparative advantage (RCA) indices using the value added categories obtained from the trade accounting framework. Results are saved in `data/`.

For country $s$ and sector $i$, the basic formula of the RCA index is:

$$
{RCA}_{(s,i)} = \frac{{Exports}_{(s,i)}/{Exports}_s}{\sum_r {Exports}_{(r,i)}/{Exports}_r}
$$

Exports may be replaced by value added exports to get a value-added-adjusted RCA.

# Set up

In [1]:
import pandas as pd
import duckdb

In [2]:
ta, output = 'ta.parquet', 'rca.parquet'
# ta, output = 'ta62.parquet', 'rca62.parquet'
# ta, output = 'ta62-const.parquet', 'rca62-const.parquet'

## Calculate RCA indices

### Export sector breakdown

In [3]:
df_es = duckdb.sql(
    f"""
    (
        SELECT 
            table_si.t, table_si.s, 5 AS agg, table_si.i5 AS i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i5, sum(Exports) AS exports_si, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s, i5
        ) AS table_si

        JOIN (
            SELECT t, s, sum(Exports) AS exports_s, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i5, sum(Exports) AS exports_i, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, i5
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i5 = table_i.i5

        JOIN (
            SELECT t, sum(Exports) AS exports_all, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i5, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i5
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 15 AS agg, table_si.i15 AS i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i15, sum(Exports) AS exports_si, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s, i15
        ) AS table_si

        JOIN (
            SELECT t, s, sum(Exports) AS exports_s, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i15, sum(Exports) AS exports_i, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, i15
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i15 = table_i.i15

        JOIN (
            SELECT t, sum(Exports) AS exports_all, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i15, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i15
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 35 AS agg, table_si.i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i, sum(Exports) AS exports_si, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s, i
        ) AS table_si

        JOIN (
            SELECT t, s, sum(Exports) AS exports_s, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i, sum(Exports) AS exports_i, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t, i
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i = table_i.i

        JOIN (
            SELECT t, sum(Exports) AS exports_all, sum(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='es'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i
    )
    """
).df()

### Origin sector breakdown

In [4]:
df_os = duckdb.sql(
    f"""
    (
        SELECT 
            table_si.t, table_si.s, 5 AS agg, table_si.i5 AS i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i5, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s, i5
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i5, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, i5
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i5 = table_i.i5

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i5, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i5
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 15 AS agg, table_si.i15 AS i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i15, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s, i15
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i15, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, i15
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i15 = table_i.i15

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i15, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i15
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 35 AS agg, table_si.i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s, i
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t, i
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i = table_i.i

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/{ta}') WHERE breakdown='os'
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i
    )
    """
).df()

### Consolidate and export

In [5]:
df = pd.merge(df_es, df_os)

df['rca'] = (df['exports_si'] / df['exports_s']) / (df['exports_i'] / df['exports_all'])
df['rca_vaxes'] = (df['vaxes_si'] / df['vaxes_s']) / (df['vaxes_i'] / df['vaxes_all'])
df['rca_vaxos'] = (df['vaxos_si'] / df['vaxos_s']) / (df['vaxos_i'] / df['vaxos_all'])

df.to_parquet(f'../data/{output}', index=False)

### View results

In [6]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()

,t,s,agg,i,exports_si,exports_s,exports_i,exports_all,vaxes_si,vaxes_s,vaxes_i,vaxes_all,vaxos_si,vaxos_s,vaxos_i,vaxos_all,rca,rca_vaxes,rca_vaxos
0,2017,1,5,1,180310.159417,3.070807e+05,2.322732e+06,2.302460e+07,154467.694950,2.619805e+05,1.893080e+06,1.624261e+07,112307.231852,2.619805e+05,2.280682e+06,1.624261e+07,5.820506,5.058893,3.053022
1,2017,1,5,2,26439.396307,3.070807e+05,3.720893e+06,2.302460e+07,22347.418348,2.619805e+05,2.678975e+06,1.624261e+07,24655.796688,2.619805e+05,2.126256e+06,1.624261e+07,0.532775,0.517185,0.718936
2,2017,1,5,3,27225.291161,3.070807e+05,1.031134e+07,2.302460e+07,19680.367675,2.619805e+05,6.415289e+06,1.624261e+07,15452.708070,2.619805e+05,4.158495e+06,1.624261e+07,0.197969,0.190197,0.230386
3,2017,1,5,4,62680.560532,3.070807e+05,6.181768e+06,2.302460e+07,55929.142685,2.619805e+05,4.846608e+06,1.624261e+07,97586.612730,2.619805e+05,7.036047e+06,1.624261e+07,0.760256,0.715463,0.859901
4,2017,1,5,5,10425.274551,3.070807e+05,4.878687e+05,2.302460e+07,9555.891735,2.619805e+05,4.086592e+05,1.624261e+07,11978.166053,2.619805e+05,6.411310e+05,1.624261e+07,1.602227,1.449762,1.158325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24085,2022,73,35,31,18213.907000,2.813121e+06,1.090159e+05,3.083029e+07,14340.071177,1.935598e+06,9.373389e+04,2.149884e+07,19050.207753,1.935598e+06,1.927204e+05,2.149884e+07,1.831060,1.699239,1.097923
24086,2022,73,35,32,13170.327289,2.813121e+06,7.052394e+04,3.083029e+07,10725.876662,1.935598e+06,6.360418e+04,2.149884e+07,9912.525647,1.935598e+06,1.079524e+05,2.149884e+07,2.046677,1.873040,1.019886
24087,2022,73,35,33,9870.339282,2.813121e+06,5.252361e+04,3.083029e+07,7322.264283,1.935598e+06,4.304239e+04,2.149884e+07,7797.669809,1.935598e+06,5.895344e+04,2.149884e+07,2.059524,1.889509,1.469114
24088,2022,73,35,34,11296.663394,2.813121e+06,3.313674e+05,3.083029e+07,8179.408026,1.935598e+06,2.731639e+05,2.149884e+07,24814.201623,1.935598e+06,3.969857e+05,2.149884e+07,0.373620,0.332582,0.694265
